In [2]:
pip install trimesh

In [3]:
import os
import random
import glob
import trimesh
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import scipy 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv3D, Dense ,Dropout, Flatten, Activation, MaxPooling3D, Input


In [4]:
try:
    from google.colab import drive
    drive.mount('/content/drive')

    print("reading from cloud...")
    path = "/content/drive/MyDrive/Colab Notebooks/MPPE/data_70p_overhang/stls_opt/"
    path_vox = "/content/drive/MyDrive/Colab Notebooks/MPPE/data_70p_overhang/voxels/"
    
except:
    import pathlib
    print("reading from disk")
    path = str(pathlib.Path().resolve())+"/dataset/stls_opt/"
    path_vox = str(pathlib.Path().resolve())+"/dataset/voxels/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
reading from cloud...


In [4]:
mesh = trimesh.load(path+"63_8475_10313_17790_28845.stl")

In [5]:
def calculate_cost(filename):
  #params
  #time - parts usally need 15 minutes for preparation
  #1h - 10eur
  price_per_h = 2
  price_per_meter = 0.17#0.1133 #150 meters costs 17e 
  filament_diameter = 2.85 #mm
  filament_phi = np.power(filament_diameter/2,2)*np.pi #mm^2
  
  support_removal_multiplier= 2
  support_removal_multiplier_time = 1.5
  profit_margin = 0.1
  #larger parts need longer time + 5 minutes slice time. 
  
  #get data from filename
  data = filename.replace(".","_").split("_")
  data = [int(x) for x in data[:-1]]

  #calculate cost from filament lenght

  #calulate cost no support 
  cost_filament = ((data[2]/(filament_phi*1000)))*price_per_meter

  #calulcate cost of printing supports only 
  cost_supports = (abs(data[2]-data[4])/(filament_phi*1000))*price_per_meter
  cost_supports_removal = support_removal_multiplier*cost_supports

  #calulate timewise cost in hours
  print_time = data[1]
  print_time_supports = data[3]

  #convert to horus and include time for removing supports 
  print_time= (((print_time + 2*abs(print_time-print_time_supports))*support_removal_multiplier_time)/3600)
  print_time_cost = print_time * price_per_h

  #cena
  cost = (cost_filament+cost_supports+cost_supports_removal+print_time_cost)
  cost = (cost*profit_margin)+cost

  return cost

#cost = calculate_cost("89_88123_134991_106833_152824.stl")
#cost = calculate_cost("62_1605_2129_2398_3618.stl")
cost = calculate_cost("108_31956_48635_34435_50936.stl")
print(cost)

35.46582523917991


In [21]:
X_train = np.zeros([0,100,100,100])
X_train_volume = np.zeros([0,1])
X_train_area = np.zeros([0,1])

y_train = np.zeros(0)
for filename in os.listdir(path_vox):
    if filename.endswith(".npy"):
        #print("appending", filename.split(".")[0])
        vox = np.load(path_vox+filename)
        X_train = np.append(X_train,vox[np.newaxis,...],axis=0)
        y_train = np.append(y_train,calculate_cost(filename))
        
        mesh = trimesh.load(path+filename.split(".")[0]+".stl")
        
        mv = np.array(abs(mesh.volume))
        X_train_volume = np.append(X_train_volume,mv[np.newaxis,...])

        ma = np.array(abs(mesh.area))
        X_train_area = np.append(X_train_area,ma[np.newaxis,...])

X_train_area = X_train_area[...,np.newaxis]
X_train_volume = X_train_volume[...,np.newaxis]
X_train = X_train[...,np.newaxis]

In [27]:
#my net
input1 = keras.Input(shape=(X_train.shape[1:]))
input2 = keras.Input(shape=(X_train_volume.shape[1:]))
input3 = keras.Input(shape=(X_train_area.shape[1:]))

#y = Dense(1)(input2)
activ = "LeakyReLU"
x = Conv3D(64,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(input1)
x = Conv3D(64,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
x = layers.Dropout(0.3)(x)
x = Conv3D(32,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
x = Conv3D(32,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
x = layers.Dropout(0.3)(x)
x = Conv3D(16,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
x = Conv3D(16,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
x = MaxPooling3D(pool_size=(1, 1, 1))(x)
x = Conv3D(8,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)
x = Conv3D(8,(3,3,3),strides = (2,2,2), activation=activ, padding = 'same')(x)

x = Activation('relu')(x)

x = layers.Dropout(0.5)(x)

x = Flatten()(x)

x = layers.Concatenate(axis=1)([x,input2])
#x = Dense(4, activation = 'relu')(x)
x = layers.Concatenate(axis=1)([x,input3])

dense = Dense(2000, activation = activ)(x)
dense = Dense(300, activation = activ)(dense)
dense = Dense(150, activation = activ)(dense)
dense = Dense(20, activation = activ)(dense)
dense = Dense(16, activation = activ)(dense)
    # final layer with 10 neurons to classify the instances
output = Dense(1, activation = 'linear')(dense)
    
#outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=[input1,input2,input3], outputs=output, name="jjnet")

In [38]:
#@title Default title text
#paper net 
input1 = keras.Input(shape=(X_train.shape[1:]))
input2 = keras.Input(shape=(X_train_volume.shape[1:]))
input3 = keras.Input(shape=(X_train_area.shape[1:]))



x = Conv3D(16,(3,3,3), activation='LeakyReLU', padding = 'same')(input1)
x = Conv3D(16,(3,3,3), activation='LeakyReLU', padding = 'same')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
x = layers.Dropout(0.3)(x)

x = Conv3D(32,(3,3,3), activation='LeakyReLU', padding = 'same')(x)
x = Conv3D(32,(3,3,3), activation='LeakyReLU', padding = 'same')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
x = layers.Dropout(0.3)(x)

x = Conv3D(8,(3,3,3), activation='LeakyReLU', padding = 'same')(x)

x = layers.Dropout(0.8)(x)

x = Flatten()(x)

x = layers.Concatenate(axis=1)([x,input2])
x = Dense(8, activation = 'LeakyReLU')(x)
dense = Dense(2000, activation = 'LeakyReLU')(x)
dense = Dense(300, activation = 'LeakyReLU')(dense)
dense = Dense(150, activation = 'LeakyReLU')(dense)
dense = Dense(20, activation = 'LeakyReLU')(dense)
dense = Dense(16, activation = 'LeakyReLU')(dense)
    # final layer with 10 neurons to classify the instances
output = Dense(1, activation = 'linear')(dense)
    
#outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=[input1,input2], outputs=output, name="papernet")

In [36]:
tf.keras.backend.clear_session()

In [ ]:
y_train_max = y_train.max()

X_train_volume_max = X_train_volume.max()

model.compile(
    loss="MAE",
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["MAPE"],
)

model.fit([X_train,np.log(X_train_volume), np.log(X_train_area)], np.log(y_train), batch_size = 8, epochs = 100, verbose = 1,validation_split=0.2)


Epoch 1/100
19/19 [==============================] - 6s 264ms/step - loss: 2.8345 - MAPE: 123.3151 - val_loss: 2.4768 - val_MAPE: 101.0383
Epoch 2/100
19/19 [==============================] - 5s 243ms/step - loss: 2.3649 - MAPE: 109.0808 - val_loss: 2.0527 - val_MAPE: 105.7807
Epoch 3/100
19/19 [==============================] - 5s 243ms/step - loss: 2.0299 - MAPE: 122.6528 - val_loss: 1.7732 - val_MAPE: 113.5100
Epoch 4/100
19/19 [==============================] - 5s 243ms/step - loss: 1.7425 - MAPE: 134.8528 - val_loss: 1.5258 - val_MAPE: 124.4967
Epoch 5/100
19/19 [==============================] - 5s 244ms/step - loss: 1.4727 - MAPE: 157.1283 - val_loss: 1.2519 - val_MAPE: 138.4190
Epoch 6/100
19/19 [==============================] - 5s 243ms/step - loss: 1.1871 - MAPE: 194.4928 - val_loss: 1.0278 - val_MAPE: 160.0084
Epoch 7/100
19/19 [==============================] - 5s 244ms/step - loss: 0.9950 - MAPE: 217.4267 - val_loss: 0.9230 - val_MAPE: 180.3267
Epoch 8/100
19/19 [========

In [23]:
y_train_max = y_train.max()
preds = model.predict([X_train, np.log(X_train_volume),np.log(X_train_area)])#*y_train_max

In [24]:
np.sum(((preds)-(np.log(y_train))))/len(preds)

-18.861018276527652

In [25]:
for e,i in zip(np.exp(preds),y_train):
  print("preds",e,"truth",i)

preds [16.324484] truth 26.77824824484932
preds [17.028578] truth 81.16618971043529
preds [43.010803] truth 51.93106089008093
preds [11.360003] truth 10.495122867463099
preds [11.359909] truth 6.806431057703429
preds [4.694588] truth 5.150449286780538
preds [29.350004] truth 38.24498913580695
preds [5.663525] truth 5.754845714363635
preds [62.525017] truth 108.79986700316321
preds [3.5530653] truth 2.656544727169499
preds [5.3706226] truth 5.788570329257202
preds [6.558576] truth 6.800669008657024
preds [19.91612] truth 36.266246189857625
preds [16.153032] truth 14.696273923049322
preds [10.7199745] truth 11.33614703354167
preds [3.7820935] truth 7.426233928909042
preds [1.9978192] truth 1.3101400525558313
preds [51.932774] truth 70.797070671036
preds [12.320688] truth 11.014982664895943
preds [3.9253225] truth 3.41721456384545
preds [17.615437] truth 46.65956068510782
preds [19.962296] truth 18.833951718741364
preds [0.66393834] truth 0.16715615662385272
preds [14.638321] truth 7.7594

In [ ]:
model.summary()

Model: "jjnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 100, 100, 1  0           []                               
                                00, 1)]                                                           
                                                                                                  
 conv3d_35 (Conv3D)             (None, 50, 50, 50,   896         ['input_15[0][0]']               
                                32)                                                               
                                                                                                  
 conv3d_36 (Conv3D)             (None, 25, 25, 25,   27680       ['conv3d_35[0][0]']              
                                32)                                                           